## HS.3.1a

In [ ]:
def reflection_matrix(main_wires, aux_wires):
    """
    Builds the reflection matrix needed for qubitization.

    Args:
    - main_wires (list(int)): The wires that make up the main register.
    - aux_wires (list(int)): The wires that make up the auxiliary register.
    """
    
    ####################
    ###YOUR CODE HERE###
    ####################
    
    # na = len(aux_wires)
    # nm = len(main_wires)
    # nsa, nsm = 2**na, 2**nm
    # aux_pos = np.zeros((nsa, nsa))
    # aux_pos[0][0] = 2
    # pos = np.kron(aux_pos, np.eye(nsm))
    # neg = np.eye(nsa*nsm)

    # return pos - neg

    matrix = qml.matrix(2 * qml.Projector(state,aux_wires) @ qml.Identity(main_wires)  - qml.Identity(aux_wires + main_wires))

    return matrix



## HS.3.1b

In [ ]:
dev = qml.device("default.qubit")

@qml.qnode(dev)
def my_qubitization(main_wires, aux_wires, state, hamiltonian):
    """
    Using the reflection matrix in the previous exercise and the built-in 
    PrepSelPrep, this function implements the Qubitization routine to a 
    Hamiltonian. The initial state in the main register is also specified.

    Args:
    - main_wires (list(int)): The main register wires in the PrepSelPrep routine.
    - aux_wires (list(int)): The auxiliary register wires in the PrepSelPrep routine.
    - state (list(float)): The initial state in the main register.
    - hamiltonian (pennylane.operator): The Hamiltonian operator that we want to encode.

    Returns
    - the output quantum state
    """


    ####################
    ###YOUR CODE HERE###
    ####################
    qml.StatePrep(state, wires=main_wires)
    qml.QubitUnitary(reflection_matrix(main_wires, aux_wires), wires=(aux_wires + main_wires))
    qml.PrepSelPrep(hamiltonian, aux_wires)

    return qml.state()



## HS.3.2a

In [ ]:
H = qml.Hamiltonian([0.5, 0.125, 0.25], [qml.Z(0) @ qml.Z(1), qml.Z(0), qml.Z(1)])# Create the Hamiltonian here

control_wires = [2, 3]
estimation_wires = [4, 5, 6, 7, 8 ,9]

dev = qml.device("default.qubit")

@qml.qnode(dev)
def prep_sel_prep_qpe(state):
    """
    Applies QPE to the PrepSelPrep encoding for the Hamiltonian H with some initial ground state

    Args:
    - state (list(float)): The initial state ground state candidate
    Returns:
    - np.ndarray(float): The output probabilities
    """

    ####################
    ###YOUR CODE HERE###
    ####################
    qml.StatePrep(np.array(state), wires=[0, 1])
    PSP = qml.PrepSelPrep(H, control=control_wires)
    qml.QuantumPhaseEstimation(PSP, estimation_wires=estimation_wires)

    return qml.probs(wires=estimation_wires)

# We know the matrix is diagonal, so the set of eigenstates is the computational basis
results = [prep_sel_prep_qpe(state) for state in np.eye(4)]
lambda_ = sum([abs(coeff) for coeff in H.terms()[0]])
eigenvalues = [lambda_ * np.cos(2 * np.pi * np.argmax(result) / 2 ** (len(estimation_wires))) for result in results]

# Print the eigenvalues
print("E = ", eigenvalues)



## HS.3.2b

In [ ]:
H = qml.Hamiltonian([0.5, 0.125, 0.25], [qml.Z(0) @ qml.Z(1), qml.Z(0), qml.Z(1)])# Create the Hamiltonian here

control_wires = [2, 3]
estimation_wires = [4, 5, 6, 7, 8 ,9]

dev = qml.device("default.qubit")

@qml.qnode(dev)
def prep_sel_prep_qpe(state):
    """
    Applies QPE to the Qubitization for the Hamiltonian H with some initial ground state

    Args:
    - state (list(float)): The initial state ground state candidate
    Returns:
    - np.ndarray(float): The output probabilities
    """

    ####################
    ###YOUR CODE HERE###
    ####################
    qml.StatePrep(np.array(state), wires=[0, 1])
    Q = qml.Qubitization(H, control=control_wires)
    qml.QuantumPhaseEstimation(Q, estimation_wires=estimation_wires)

    return qml.probs(wires=estimation_wires)

# We know the matrix is diagonal, so the set of eigenstates is the computational basis
results = [prep_sel_prep_qpe(state) for state in np.eye(4)]
lambda_ = sum([abs(coeff) for coeff in H.terms()[0]])
eigenvalues = [lambda_ * np.cos(2 * np.pi * np.argmax(result) / 2 ** (len(estimation_wires))) for result in results]

# Print the eigenvalues
print("E = ", eigenvalues)
